In [1]:
import yfinance as yf
yf.pdr_override()
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
from pandas_datareader import data as pdr
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')

import math

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

C:\Users\Administrator\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
C:\Users\Administrator\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [3]:
def CAPM(CODE):
    if CODE.endswith('.KS'): # 한국 기업이면 코스피 사용
        KOSPI_CODE = '^KS11'
        RF = 3.119
        ERP = 5.36
    else:
        KOSPI_CODE = '^IXIC' # 다른 나라 기업이면 일단은 미국 기업으로 산정
        RF = 2.8741
        ERP = 4.24

    FIRST = '2017-05-08'
    TODAY = '2022-05-06' #dt.strftime(dt.today(),"%Y-%m-%d")
    INTERVAL ='1wk'

    price = yf.download(CODE, start = FIRST, end = TODAY, interval=INTERVAL, progress=False)
    kospi = yf.download(KOSPI_CODE, start = FIRST, end = TODAY, interval=INTERVAL, progress=False)

    df_stock = pd.concat([price['Close'], round(price['Close'].pct_change()*100,2)], axis=1)
    df_kospi = pd.concat([kospi['Close'], round(kospi['Close'].pct_change()*100,2)], axis=1)

    df_stock.columns = ['stock_price', 'stock_pct']
    df_stock = df_stock.dropna()

    df_kospi.columns = ['kospi_price', 'kospi_pct']
    df_kospi = df_kospi.dropna()

    df = pd.concat([df_kospi, df_stock], axis=1, join='inner',)

    y = df['stock_pct'].values
    X = df['kospi_pct'].values

    X = X.reshape(-1,1)
    y = y.reshape(-1,1)

    results = sm.OLS(y, sm.add_constant(X)).fit()
    BETA = results.params[1]

    print('Rf :',RF)
    print('ERP :', ERP)
    print("Beta :", BETA)

    rm = RF + BETA*ERP
    print('E(r) :',rm)
    print('')

In [4]:
def ROE(CODE):
    df_stock = yf.Ticker(CODE)
    ROE = df_stock.financials.loc['Net Income'] / df_stock.balancesheet.loc['Total Stockholder Equity'] 
    return ROE.drop('2018-12-31').values

def NET_INCOME(CODE):
    df_stock = yf.Ticker(CODE)
    df0 = df_stock.financials.loc['Net Income'].drop('2018-12-31').values
    df1 = list(math.floor(i/1000) for i in df0)
    return df1

def CHANGE_IN_WORKING_CAPITAL(CODE):
    df_stock = yf.Ticker(CODE)
    df1 = df_stock.balancesheet.loc['Total Current Assets'] - df_stock.balancesheet.loc['Total Current Liabilities']
    df2 = []
    for i in range(len(df1.values)-1):
        df2.append(math.floor((df1.values[i]-df1.values[i+1])/1000))
    return df2

def CHANGE_IN_DEBT(CODE):
    options = Options()
    options.add_argument('headless')
    driver = webdriver.Chrome(chrome_options=options, executable_path='./chromedriver.exe')

    wait = WebDriverWait(driver, 10)
    driver.get(f'https://finance.yahoo.com/quote/{CODE}/cash-flow?p={CODE}')
    wait

    li = []
    li2 = []
    for i in range(1,20):
        try:
            a = driver.find_element(By.XPATH, f'//*[@id="Col1-1-Financials-Proxy"]/section/div[3]/div[1]/div/div[2]/div[{i}]/div[1]/div[1]/div[1]')
            if a.text == 'Issuance of Debt':
                for j in range(3,7):
                    k = driver.find_element(By.XPATH, f'//*[@id="Col1-1-Financials-Proxy"]/section/div[3]/div[1]/div/div[2]/div[{i}]/div[1]/div[{j}]/span')
                    p = k.text.replace(',','')
                    li.append(int(p))

            if a.text == 'Repayment of Debt':
                for j in range(3,7):
                    k = driver.find_element(By.XPATH, f'//*[@id="Col1-1-Financials-Proxy"]/section/div[3]/div[1]/div/div[2]/div[{i}]/div[1]/div[{j}]/span')
                    q = k.text.replace(',','')
                    li2.append(int(q))
        except NoSuchElementException:
            break
    
    li3 = []
    for i,j in zip(li, li2):
        li3.append(i+j)

    return li3[:3]

def NET_CAPEX(CODE):    
    options = Options()
    options.add_argument('headless')
    driver = webdriver.Chrome(chrome_options=options, executable_path='./chromedriver.exe')

    wait = WebDriverWait(driver, 20)
    driver.get(f'https://finance.yahoo.com/quote/{CODE}/balance-sheet?p={CODE}')
    wait
    
    click = driver.find_element(By.XPATH,'//*[@id="Col1-1-Financials-Proxy"]/section/div[2]/button/div/span')
    click = click.text
    wait
    
    if click != 'Expand All':
        driver.find_element(By.XPATH,'//*[@id="Col1-1-Financials-Proxy"]/section/div[2]/button/div/span').click()
    wait
    
    li = []
    for i in range(1,20):
        try:
            a = driver.find_element(By.XPATH, f'//*[@id="Col1-1-Financials-Proxy"]/section/div[3]/div[1]/div/div[2]/div[1]/div[1]/div[1]/div[1]/span')
            if a.text == 'Total Assets':
                driver.find_element(By.XPATH, f'//*[@id="Col1-1-Financials-Proxy"]/section/div[3]/div[1]/div/div[2]/div[1]/div[1]/div[1]/div[1]/button/svg').click()
        
                for j in range(2,6):
                    k = driver.find_element(By.XPATH, f'//*[@id="Col1-1-Financials-Proxy"]/section/div[3]/div[1]/div/div[2]/div[{i}]/div[1]/div[{j}]/span')
                    p = k.text.replace(',','')
                    li.append(int(p))

    li = []
    for i in range(2,6):
        a = driver.find_element(By.XPATH, f'//*[@id="Col1-1-Financials-Proxy"]/section/div[3]/div[1]/div/div[2]/div[1]/div[2]/div[2]/div[2]/div[1]/div[2]/div[2]/div[1]/div[{i}]/span')
        a = a.text.replace(',','')

        li.append(int(a))
    driver.quit()
    
    df_stock = yf.Ticker(CODE)
    
    li2 = []
    for i in range(len(li)-1):
        li2.append(li[i+1] - li[i])
        
    a = (df_stock.balancesheet.loc['Total Assets'] - df_stock.balancesheet.loc['Total Current Assets'])/1000
    b = li2
    
    a1 = []
    for i in range(len(a)-1):
        a1.append(a[i] - a[i+1])

    li5 = []
    for i,j in zip(a1,b):
        li5.append(math.floor(i+j))
        
    return li5



In [ ]:
def NET_CAPEX(CODE):    
    options = Options()
    options.add_argument('headless')
    driver = webdriver.Chrome(chrome_options=options, executable_path='./chromedriver.exe')

    wait = WebDriverWait(driver, 20)
    driver.get(f'https://finance.yahoo.com/quote/{CODE}/balance-sheet?p={CODE}')
    wait
    
    click = driver.find_element(By.XPATH,'//*[@id="Col1-1-Financials-Proxy"]/section/div[2]/button/div/span')
    click = click.text
    wait
    
    if click == 'Expand All':
        driver.find_element(By.XPATH,'//*[@id="Col1-1-Financials-Proxy"]/section/div[2]/button/div/span').click()

    wait
    li = []
    for i in range(2,6):
        a = driver.find_element(By.XPATH, f'//*[@id="Col1-1-Financials-Proxy"]/section/div[3]/div[1]/div/div[2]/div[1]/div[2]/div[2]/div[2]/div[1]/div[2]/div[2]/div[1]/div[{i}]/span')
        a = a.text.replace(',','')

        li.append(int(a))
    driver.quit()
    
    df_stock = yf.Ticker(CODE)
    
    li2 = []
    for i in range(len(li)-1):
        li2.append(li[i+1] - li[i])
        
    a = (df_stock.balancesheet.loc['Total Assets'] - df_stock.balancesheet.loc['Total Current Assets'])/1000
    b = li2
    
    a1 = []
    for i in range(len(a)-1):
        a1.append(a[i] - a[i+1])

    li5 = []
    for i,j in zip(a1,b):
        li5.append(math.floor(i+j))
        
    return li5

In [5]:
def HIGH_GROWTH_MODEL(CODE):
    df = pd.DataFrame(columns=['Net CAPEX','Change In Working Capital', 'change in debt', 'Net Income', 'b', 'ROE', 'Equity RIR'],
                     index = ['2021','2020','2019'])
    # Net CAPEX
    #print('Net Capex')
    df['Net CAPEX'] = NET_CAPEX(CODE)
    
    # Change In Working Capital
    #print("Change in working Capital")
    df['Change In Working Capital'] = CHANGE_IN_WORKING_CAPITAL(CODE)
    
    # Change In Debt
    #print("Change in Debt")
    df['change in debt'] = CHANGE_IN_DEBT(CODE)
    
    # Net Inocme
    #print("Net Income")
    df['Net Income'] = NET_INCOME(CODE)
    
    # b
    #print("b")
    df['b'] = (df['Net CAPEX'] + df['Change In Working Capital'] - df['change in debt'])/df['Net Income']
    
    # ROE
    #print("ROE")
    df['ROE'] = ROE(CODE)
    
    display(df)
    return df


In [7]:
CODES = ['VZ','042700.KS','005930.KS','006400.KS'] # 원하는 기업 데이터

for CODE in CODES:
    print(f"-------- {CODE} - CAPM ----------")
    CAPM(CODE)

    print(f"-------- {CODE} - HIGH_GROWTH_MODEL ----------")
    HIGH_GROWTH_MODEL(CODE)

print("---------------- END ----------------")

-------- VZ - CAPM ----------
Rf : 2.8741
ERP : 4.24
Beta : 0.2559825056074729
E(r) : 3.959465823775685

-------- VZ - HIGH_GROWTH_MODEL ----------


Net CAPEX  Change In Working Capital  change in debt  Net Income  \
2021   73278000                  -25366000        22554000    22065000   
2020   18718000                   22329000        14269000    17801000   
2019   34331000                   -4101000        -5231000    19265000   

             b       ROE Equity RIR  
2021  1.149241  0.269776        NaN  
2020  1.504298  0.262389        NaN  
2019  1.840696  0.313788        NaN

-------- 042700.KS - CAPM ----------
Rf : 3.119
ERP : 5.36
Beta : 1.1513007916091724
E(r) : 9.289972243025165

-------- 042700.KS - HIGH_GROWTH_MODEL ----------


Net CAPEX  Change In Working Capital  change in debt  Net Income  \
2021   65085025                   27727490               0   104438304   
2020   -4942902                   43630072               0    50131484   
2019   -5064898                   13605890               0    19262570   

             b       ROE Equity RIR  
2021  0.888683  0.301138        NaN  
2020  0.771714  0.194995        NaN  
2019  0.443398  0.087675        NaN

-------- 005930.KS - CAPM ----------
Rf : 3.119
ERP : 5.36
Beta : 1.087963659887095
E(r) : 8.95048521699483

-------- 005930.KS - HIGH_GROWTH_MODEL ----------


Net CAPEX  Change In Working Capital  change in debt   Net Income  \
2021  56327868000                 7434824000     -3453413000  39243791000   
2020  24244028000                 5008732000      1340734000  26090846000   
2019  27028270000                11986582000       156392000  21505054000   

             b       ROE Equity RIR  
2021  1.712783  0.132474        NaN  
2020  1.069801  0.097474        NaN  
2019  1.806945  0.084362        NaN

-------- 006400.KS - CAPM ----------
Rf : 3.119
ERP : 5.36
Beta : 1.3547386070662817
E(r) : 10.38039893387527

-------- 006400.KS - HIGH_GROWTH_MODEL ----------


Net CAPEX  Change In Working Capital  change in debt  Net Income  \
2021  3497337297                  309849249       648289028  1169801395   
2020  1969023929                 -766120223       330689399   574723494   
2019  1429389292                  -66627466       317286098   356548861   

             b       ROE Equity RIR  
2021  2.700371  0.079555        NaN  
2020  1.517624  0.044286        NaN  
2019  2.932209  0.028929        NaN

---------------- END ----------------
